In [30]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import regex as re

Fil = pd.read_pickle('../data/raw/Fil.pkl')

In [24]:
Fil.iloc[0]

id                                                               158
dokumentid                                                     11432
titel              Evaluering af lov om friplejeboliger_endelig v...
versionsdato                                     2014-09-04 15:44:54
filurl             https://www.ft.dk/samling/20131/almdel/BYB/bil...
opdateringsdato                           2020-02-25 10:46:59.490000
variantkode                                                        P
format                                                           PDF
Name: 0, dtype: object

In [8]:
test = Fil.iloc[0]['filurl']
test

'https://www.ft.dk/samling/20131/almdel/BYB/bilag/89/1396405.pdf'

In [9]:
test = test[:-4]+'/index.htm'
test

'https://www.ft.dk/samling/20131/almdel/BYB/bilag/89/1396405/index.htm'

In [26]:
r = requests.get(test)
print(r.text)

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN">
<!-- saved from url=(0052)http://www.verypdf.com/ -->
<html>
<head>
<meta name="keywords" content="Til
Ministeriet for By, Bolig og Landdistrikter
Dato
Januar 2012
Version
1.0
EVALUERING AF LOV OM FRIPLEJEBOLIGER

">
<TITLE>PDF to HTML - Convert PDF files to HTML files</TITLE>
<meta name="description" content="PDF to HTML - Convert PDF files to HTML files">
<meta name="keywords" content="PDF to HTML - Convert PDF files to HTML files">
<meta name="resource-type" content="document">
<meta name="Reply-to" content="support@verypdf.com">
<meta name="expires" content="never">
<meta name="distribution" content="Global">
<meta name="robots" content="INDEX,FOLLOW">
<meta name="email" content="support@verypdf.com">
<meta name="publisher" content="VeryPDF.com Inc">
<meta name="copyright" content="Copyright (C) 2002-2007 VeryPDF.com Inc">
<meta name="author" content="VeryPDF.com Inc">
<meta name="classification" content="General Softwa

In [37]:
def extract_text_from_html(link):
    #requesting the html document
    html_doc = requests.get(link).text

    soup = BeautifulSoup(html_doc, 'html.parser')
    return soup.get_text()

In [38]:
text = extract_text_from_html(test)
#removing the sentence "PDF to HTML - Convert PDF files to HTML files"
text = re.sub(r'PDF to HTML - Convert PDF files to HTML files', '', text)
#removing newlines when there are more than 2 in a row
text = re.sub(r'\n{3,}', '\n\n', text)
print(text)




By- og Boligudvalget 2013-14BYB Alm.del Bilag 89Offentligt

Til
Ministeriet for By, Bolig og Landdistrikter
Dato
Januar 2012
Version
1.0
EVALUERING AF LOV OM
FRIPLEJEBOLIGER

EVALUERING AF LOV OM FRIPLEJEBOLIGER
Rambøll
Hannemanns Allé 53
DK-2300 København S
T +45 5161 1000
F +45 5161 1001
www.ramboll-management.dk

EVALUERING AF LOV OM FRIPLEJEBOLIGER
INDHOLD
1.
1.1
1.2
1.3
2.
2.1
2.2
2.3
2.4
3.
3.1
3.2
3.2.1
3.2.2
3.2.3
3.2.4
3.2.5
4.
4.1
4.2
4.2.1
4.2.2
4.2.3
4.3
4.3.1
4.3.2
4.3.3
4.4
4.5
4.6
5.
5.1
5.2
5.3
5.4
5.5
5.6
5.7
5.8
5.9
6.
6.1
6.2
6.2.1
6.2.2
6.2.3
6.3
Indledning
Formål
Evalueringsdesign
Rapportens struktur og læsevejledning
Resumé
Lovgivningens indretning
Borgernes anvendelse af det frie valg
Konkurrencesituationen
Kommunernes administration
Lovgivningens formål og indhold
Lovens formål
Lovens indhold
Certificering
Etablering
Drift og afregning af service og pleje
Tilsyn
Udbud og afhændelse
Implementeringen af loven
Udnyttelse af kvoter
Leverandørerne af friplejebolige

In [39]:
paragraphs

<body bgcolor="#ffffff" link="#FFFFFF" vlink="#FFFFFF"><div class="manchet">By- og Boligudvalget 2013-14<br/>BYB Alm.del Bilag 89<br/>Offentligt<br/></div>
<script type="text/javascript">
var currentpos,timer; 
function initialize() 
{ 
timer=setInterval("scrollwindow()",10);
} 
function sc(){
clearInterval(timer); 
}
function scrollwindow() 
{ 
currentpos=document.body.scrollTop; 
window.scroll(0,++currentpos); 
if (currentpos != document.body.scrollTop) 
sc();
} 
document.onmousedown=sc
document.ondblclick=initialize
</script>
<nobr>
<div class="pos" id="*0:0" style="top:0;left:0"><img alt="1396405_0001.png" height="1100" name="1396405_0001.png" src="/pimage/TingDok_Archive/2014_09/1396405/1396405_0001.png" width="850"/></div>
<div class="pos" id="*119:113" style="top:119;left:113"><span id="*8" style="font-style:normal;font-weight:normal;font-size:8px;font-family:Verdana;color:#000000;">Til</span></div>
<div class="pos" id="*132:113" style="top:132;left:113"><span id="*10" style="fo